Imports

In [127]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import statsmodels.api as sm
import os
from datetime import datetime

# Influencing factors in media coverage

Our hypothesis is that we can observe come connection between the volume of the appearance of a disaster in the media and the characteristics of the disasters. To be concrete we would expect that there are more quotes about disasters with more casualities. In this section our aim is to determine these factors in the observed data of diasaters and examine their contribution to media coverage.

### Earthquakes in Quotebank

Previously, we filtered the quotes in the quotebank, so our task is now to connect them with the rigth earthquake. 

In [102]:
# import the quotes
files = os.listdir("earthquake_dataset_labels")


quotes_eq = pd.DataFrame()

for file in files: 
    e = pd.read_csv("earthquake_dataset_labels/"+file)
    e["Dis No"] = file[:-4]
    quotes_eq = quotes_eq.append(e)

quotes_eq.head()

,Unnamed: 0,quoteID,quotation,speaker,qids,date,numOccurrences,urls,phase,speaker_prob,url_countries,Dis No
0,209035,2009-04-06-039462,it was with great sadness that i learned of th...,jose manuel barroso,Q15849,2009-04-06 14:08:17,2,['http://abc.net.au/news/stories/2009/04/06/25...,A,0.2864,['GR' 'AS'],2009-0136-ITA
1,90671,2009-07-07-066375,there would have to be an earthquake of a stre...,guido bertolaso,Q3119409,2009-07-07 11:04:36,31,['http://ktla.com/news/nationworld/sns-ap-g8-s...,A,0.2685,['IN' 'US' 'CA'],2009-0136-ITA
2,28938,2009-04-08-093404,"when the earthquake hit, i was staying with my...",francesco de angelis,Multiple,2009-04-08 19:42:24,1,['http://pubs.acs.org/cen/news/87/i15/8715notw...,A,0.3876,['US'],2009-0136-ITA
3,9545,2009-04-18-053784,"we do know that some earthquakes, including th...",rick sanchez,Q6091403,2009-04-18 08:06:35,1,['http://ajr.org/Article.asp?id=4751'],A,0.3884,['US'],2009-0136-ITA
4,17987,2009-04-06-068877,the tremors being felt by the population are p...,None,NaN,2009-04-06 11:52:02,35,['http://thestar.com/News/World/article/614310...,A,0.3179,['NZ' 'UP' 'UK' 'SF' 'CA' 'US' 'QA' 'AS'],2009-0136-ITA


In [99]:
# TODO: Some initial analysis for these quotes

## Reaching the disasters

As we have seen in the previous milestone, we can assume that to a specific disaster the quote realted to it will contain the name of the place and the type of the disaster. So our first goal is to assign the specific quotes according to this assumption to the adequate events.

Step 1: Collect the names and places of the biggest disasters.

In [107]:
disasters = pd.read_csv("datasets/emdat/emdat_public_2021_11_06_clean.csv")
dis_num = disasters.shape[0]
disasters.head()

,Unnamed: 0,Dis No,Disaster Type,Disaster Subtype,Event Name,Country,ISO,Continent,OFDA Response,Appeal,...,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),human_readable_name,start_date,end_date
0,0,2010-0017-HTI,Earthquake,Ground movement,NaN,Haiti,HTI,Americas,True,NaN,...,300000.0,3400000.0,NaN,3700000.0,11500000.0,200000.0,8000000.0,2010 Earthquake in Haiti,2010-01-01,2010-12-31
1,5,2010-0372-RUS,Extreme temperature,Heat wave,NaN,Russian Federation (the),RUS,Europe,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,400000.0,2010 Extreme temperature in Russian Federatio...,2010-01-01,2010-12-31
2,10,2010-9082-SOM,Drought,Drought,NaN,Somalia,SOM,Africa,False,Yes,...,NaN,4000000.0,NaN,4000000.0,NaN,NaN,NaN,2010 Drought in Somalia,2010-01-01,2011-12-31
3,11,2011-0082-JPN,Earthquake,Tsunami,NaN,Japan,JPN,Asia,True,NaN,...,5933.0,362887.0,NaN,368820.0,NaN,37500000.0,210000000.0,2011 Earthquake in Japan,2011-01-01,2011-12-31
4,16,2015-0144-NPL,Earthquake,Ground movement,NaN,Nepal,NPL,Asia,True,NaN,...,17932.0,5621790.0,NaN,5639722.0,NaN,100000.0,5174000.0,2015 Earthquake in Nepal,2015-01-01,2015-12-31


## Determine the contibuting factors

We should find first the relevant columns in the disaster dataset which can be tested that it has an effect of the volume numbers. 

## WARNING 

To run the below cell, it takes hours, so dont run it if you dont want to recalculate it, you can find the data in a pickle.

In [106]:
# we have a dataset about the disasters with word developpment indicators
dis_wdi = pd.read_csv('wdi_with_disasters.csv')

indicators = dis_wdi['indicator_name'].unique()

emp = np.empty(dis_num)
emp[:] = np.NaN

for indi in indicators: 
    disasters[f'{indi}'] = emp


for i in tqdm(range(dis_num)): # iterate over disasters
    
    for indi in indicators: # iterate over indicators
     
        curr_dis = disasters.iloc[i]
        name = curr_dis['human_readable_name']
        year = int(curr_dis['start_date'][:4])

        val = dis_wdi[(dis_wdi['human_readable_name']==name) & (dis_wdi['indicator_name']==indi)][str(year)].values

        if len(val) == 0:
            pass
        else: 
            disasters.loc[disasters['human_readable_name']==name, f'{indi}'] = val[0]

            

disasters.head()

C:\Users\jhild\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 4133/4133 [1:08:43<00:00,  1.00it/s]


,Unnamed: 0,Dis No,Disaster Type,Disaster Subtype,Event Name,Country,ISO,Continent,OFDA Response,Appeal,...,"School enrollment, primary (% gross)","School enrollment, primary and secondary (gross), gender parity index (GPI)","School enrollment, secondary (% gross)",Statistical Capacity Score (Overall Average) (scale 0 - 100),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %)
0,0,2010-0017-HTI,Earthquake,Ground movement,NaN,Haiti,HTI,Americas,True,NaN,...,NaN,NaN,NaN,42.222222,27750.0,NaN,NaN,134.0,12.512662,3.618578
1,5,2010-0372-RUS,Extreme temperature,Heat wave,NaN,Russian Federation (the),RUS,Europe,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,2010-9082-SOM,Drought,Drought,NaN,Somalia,SOM,Africa,False,Yes,...,NaN,NaN,NaN,26.666667,637660.0,NaN,NaN,NaN,NaN,7.073623
3,11,2011-0082-JPN,Earthquake,Tsunami,NaN,Japan,JPN,Asia,True,NaN,...,NaN,NaN,NaN,NaN,377955.0,NaN,NaN,NaN,NaN,0.097377
4,16,2015-0144-NPL,Earthquake,Ground movement,NaN,Nepal,NPL,Asia,True,NaN,...,144.177673,1.05463,67.843689,72.222233,147180.0,14.686305,NaN,17.0,8.441650,2.445982


In [108]:
import pickle as pickle
with open(  f'disasters'+'.pickle', 'wb' ) as handle:
    pickle.dump(disasters, handle, protocol=pickle.HIGHEST_PROTOCOL)

#  NOW YOU CAN RUN THE CODES

In [214]:

disasters = pd.read_pickle('disasters.pickle')

disasters.head()

,Unnamed: 0,Dis No,Disaster Type,Disaster Subtype,Event Name,Country,ISO,Continent,OFDA Response,Appeal,...,"School enrollment, primary (% gross)","School enrollment, primary and secondary (gross), gender parity index (GPI)","School enrollment, secondary (% gross)",Statistical Capacity Score (Overall Average) (scale 0 - 100),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %)
0,0,2010-0017-HTI,Earthquake,Ground movement,NaN,Haiti,HTI,Americas,True,NaN,...,NaN,NaN,NaN,42.222222,27750.0,NaN,NaN,134.0,12.512662,3.618578
1,5,2010-0372-RUS,Extreme temperature,Heat wave,NaN,Russian Federation (the),RUS,Europe,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,2010-9082-SOM,Drought,Drought,NaN,Somalia,SOM,Africa,False,Yes,...,NaN,NaN,NaN,26.666667,637660.0,NaN,NaN,NaN,NaN,7.073623
3,11,2011-0082-JPN,Earthquake,Tsunami,NaN,Japan,JPN,Asia,True,NaN,...,NaN,NaN,NaN,NaN,377955.0,NaN,NaN,NaN,NaN,0.097377
4,16,2015-0144-NPL,Earthquake,Ground movement,NaN,Nepal,NPL,Asia,True,NaN,...,144.177673,1.05463,67.843689,72.222233,147180.0,14.686305,NaN,17.0,8.441650,2.445982


In [215]:
disasters.columns
unneeded_columns = ['Unnamed: 0', 'Disaster Type', 'Disaster Subtype',
       'Event Name', 'Country', 'ISO', 'Continent', 'Appeal', 'start_date', 'end_date', 'human_readable_name']
work_ds = disasters.drop(unneeded_columns, axis=1)
work_ds

,Dis No,OFDA Response,Declaration,Aid Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,Local Time,Total Deaths,...,"School enrollment, primary (% gross)","School enrollment, primary and secondary (gross), gender parity index (GPI)","School enrollment, secondary (% gross)",Statistical Capacity Score (Overall Average) (scale 0 - 100),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %)
0,2010-0017-HTI,True,NaN,NaN,7.0,Richter,18.443,-72.571,16:53,222570.0,...,NaN,NaN,NaN,42.222222,27750.0,NaN,NaN,134.0,12.512662,3.618578
1,2010-0372-RUS,False,NaN,NaN,40.0,°C,NaN,NaN,NaN,55736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-9082-SOM,False,Yes,NaN,NaN,Km2,NaN,NaN,NaN,20000.0,...,NaN,NaN,NaN,26.666667,637660.0,NaN,NaN,NaN,NaN,7.073623
3,2011-0082-JPN,True,NaN,NaN,9.0,Richter,38.297,142.373,14:46,19846.0,...,NaN,NaN,NaN,NaN,377955.0,NaN,NaN,NaN,NaN,0.097377
4,2015-0144-NPL,True,NaN,NaN,8.0,Richter,28.230,84.731,11:56,8831.0,...,144.177673,1.05463,67.843689,72.222233,147180.0,14.686305,NaN,17.0,8.441650,2.445982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,2019-0415-TCD,False,NaN,NaN,NaN,Km2,NaN,NaN,NaN,NaN,...,89.191139,0.72831,20.555040,50.000033,NaN,NaN,NaN,58.0,NaN,3.935693
4129,2019-0528-TON,False,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,...,115.448257,1.04883,94.093086,56.666667,NaN,NaN,NaN,16.0,3.309866,1.145440
4130,2019-0531-TON,False,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,...,115.448257,1.04883,94.093086,56.666667,NaN,NaN,NaN,16.0,3.309866,1.145440
4131,2019-0608-URY,False,NaN,NaN,73269.0,Km2,-33.749,-55.922,NaN,NaN,...,104.305038,1.05744,123.003822,77.777800,NaN,18.154268,NaN,6.5,NaN,0.456481


## Determine the volume of media coverage

The best ways to measure the media coverage of a disaster is to count the number of occurrences in the quotes and the length between the first and last appearance. 

In this part we will calculate these indicators.

In [216]:
files = os.listdir("earthquake_dataset_labels")


quotes_stat = pd.DataFrame()

i = -1
q = {}
for file in files: 
    i = i+1

    e = pd.read_csv("earthquake_dataset_labels/" + file)

    f ={}
    id = file[:-4]
    f["Dis No"] = id
    f['Number of Quotes'] = e['numOccurrences'].values.sum()
    e["date"] = e["date"].apply(lambda date: datetime.strptime(date[:10], '%Y-%m-%d'))
    if not (e["date"].empty): 
        f["Length of Appearance"] = ((e['date'].max().date() - e['date'].min().date()).days)
        e['delta_date'] = pd.to_numeric((e['date']-e['date'].min()).dt.days, downcast="integer")
        f["Length of Appearance in 1st Year"] = e[e['delta_date']<=366]['numOccurrences'].values.sum()
    else: 
        f["Length of Appearance"] = np.NaN
        f["Number of Quotes in 1st Year"] = np.NaN

    q[id] = list(f.values())

quotes_stat = pd.DataFrame.from_dict(q, orient = 'index',  columns=["Dis No", "Number of Quotes", "Length of Appearance", "Number of Quotes in 1st Year"])
quotes_stat.head()

,Dis No,Number of Quotes,Length of Appearance,Number of Quotes in 1st Year
2009-0136-ITA,2009-0136-ITA,439,4014.0,152.0
2009-0354-IDN,2009-0354-IDN,3,10.0,3.0
2009-0420-ASM,2009-0420-ASM,157,3729.0,129.0
2009-0420-WSM,2009-0420-WSM,157,3729.0,129.0
2009-0421-IDN,2009-0421-IDN,68,3906.0,51.0


In [219]:
work_ds = work_ds.merge(quotes_stat, on="Dis No", how='inner')
work_ds.head()

,Dis No,OFDA Response,Declaration,Aid Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,Local Time,Total Deaths,...,Statistical Capacity Score (Overall Average) (scale 0 - 100),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %),Number of Quotes,Length of Appearance,Number of Quotes in 1st Year
0,2010-0017-HTI,True,NaN,NaN,7.0,Richter,18.443,-72.571,16:53,222570.0,...,42.222222,27750.0,NaN,NaN,134.0,12.512662,3.618578,16620,4053.0,8322.0
1,2011-0082-JPN,True,NaN,NaN,9.0,Richter,38.297,142.373,14:46,19846.0,...,NaN,377955.0,NaN,NaN,NaN,NaN,0.097377,480,3295.0,247.0
2,2015-0144-NPL,True,NaN,NaN,8.0,Richter,28.230,84.731,11:56,8831.0,...,72.222233,147180.0,14.686305,NaN,17.0,8.441650,2.445982,2689,4155.0,2.0
3,2018-0352-IDN,True,Yes,NaN,8.0,Richter,-178.000,119.840,NaN,4140.0,...,85.555567,1916862.2,10.230143,5.268233,21.0,25.072428,2.345607,278,3761.0,1.0
4,2010-0169-CHN,True,NaN,NaN,7.0,Richter,33.165,96.548,07:49,2968.0,...,65.555557,9600000.7,10.212049,NaN,NaN,2.896535,3.255365,53,3088.0,32.0


In [220]:
work_ds.corr(method='pearson')

,OFDA Response,Aid Contribution,Dis Mag Value,Latitude,Longitude,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,...,Statistical Capacity Score (Overall Average) (scale 0 - 100),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %),Number of Quotes,Length of Appearance,Number of Quotes in 1st Year
OFDA Response,1.000000,0.918314,0.266856,-0.366189,-0.087267,0.184582,0.144955,0.335727,0.368448,0.310868,...,0.137275,-0.336903,0.199351,-0.198772,0.204530,0.350440,-0.085251,0.232095,0.167636,0.165428
Aid Contribution,0.918314,1.000000,0.116339,-0.492754,0.063137,0.971577,0.738133,0.875470,NaN,0.875824,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.252679,0.625801,0.902033
Dis Mag Value,0.266856,0.116339,1.000000,-0.313183,-0.270478,0.016843,0.044642,0.170450,0.738651,0.243633,...,0.068652,-0.443542,-0.318573,-0.185068,0.097055,0.342187,-0.097049,-0.006087,0.406483,-0.024154
Latitude,-0.366189,-0.492754,-0.313183,1.000000,0.002153,0.026736,0.036902,-0.140552,-0.940652,-0.199743,...,-0.246558,0.178358,0.072481,0.438685,-0.009526,-0.365753,0.109216,0.021005,-0.151954,0.037461
Longitude,-0.087267,0.063137,-0.270478,0.002153,1.000000,-0.204024,-0.250654,0.068048,-0.846573,-0.013106,...,-0.060192,0.333535,-0.159239,-0.969124,-0.195897,0.006671,0.472609,-0.160492,-0.235934,-0.227034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Total debt service (% of exports of goods, services and primary income)",0.350440,NaN,0.342187,-0.365753,0.006671,-0.026297,-0.072035,-0.178919,-0.087349,-0.089454,...,0.432564,-0.489372,0.322988,-0.733068,0.036080,1.000000,-0.020410,0.058056,0.344864,-0.030593
Urban population growth (annual %),-0.085251,NaN,-0.097049,0.109216,0.472609,0.221759,0.323749,0.372049,-0.359660,0.281843,...,-0.173243,0.409423,-0.878040,-0.427424,0.457613,-0.020410,1.000000,0.208829,-0.149713,0.225460
Number of Quotes,0.232095,0.252679,-0.006087,0.021005,-0.160492,0.891049,0.888653,0.366077,0.027168,0.386706,...,-0.364664,-0.241889,0.521434,-0.035045,0.720377,0.058056,0.208829,1.000000,0.273240,0.897642
Length of Appearance,0.167636,0.625801,0.406483,-0.151954,-0.235934,0.127823,0.150166,0.123897,0.215724,0.153243,...,-0.116908,-0.667128,0.375762,-0.726222,0.020669,0.344864,-0.149713,0.273240,1.000000,0.121938


In [130]:
model = sm.OLS(work_ds['Urban population growth (annual %)'], work_ds['Number of Quotes in 1st year'])
results = model.fit()
print(results.summary())

                                         OLS Regression Results                                        
Dep. Variable:     Urban population growth (annual %)   R-squared (uncentered):                     nan
Model:                                            OLS   Adj. R-squared (uncentered):                nan
Method:                                 Least Squares   F-statistic:                                nan
Date:                                Sun, 12 Dec 2021   Prob (F-statistic):                         nan
Time:                                        14:55:51   Log-Likelihood:                             nan
No. Observations:                                4133   AIC:                                        nan
Df Residuals:                                    4132   BIC:                                        nan
Df Model:                                           1                                                  
Covariance Type:                            nonrobust           

## Visualisations

This is the right place to make some nice visualizations for the datastory. 